# 44-大数据处理技术

## 📚 用途说明

**学习目标**：
- 掌握大数据处理的基本概念和架构
- 熟练使用分布式计算框架和工具
- 理解大数据存储和处理技术
- 能够构建可扩展的数据处理流水线

**前置要求**：
- 已完成43-时间序列分析学习
- 熟练掌握Python数据处理基础
- 了解基本的并行编程概念

**与LangChain关联**：
- 大数据处理支持LangChain的海量文本数据处理
- 为LangChain的分布式推理提供技术基础
- 支持LangChain的大规模知识库管理
- 确保LangChain应用的高性能和可扩展性

---

## 🔢 知识点覆盖

### 5.8 大数据处理技术 [⭐⭐进阶]
**知识点说明**：大数据处理技术是处理海量数据的核心能力，包括分布式计算、流式处理、数据存储等。掌握这些技能对于构建企业级LangChain应用非常重要。

**学习要求**：
- 掌握大数据处理的基本概念
- 理解分布式计算原理
- 熟练使用大数据处理工具
- 能够设计可扩展的数据处理架构

**案例要求**：
- 实现完整的大数据处理流程
- 进行分布式计算的性能优化
- 应用大数据技术解决实际问题
- 验证点：能独立构建有效的大数据处理系统

In [ ]:
print("🔥 大数据处理技术:")
print("=" * 50)

# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Any, Optional, Union, Dict, Iterator
from dataclasses import dataclass
import json
import time
import threading
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# 大数据处理相关库
import itertools
import collections
from functools import reduce, partial
import queue
import heapq
import hashlib
import pickle
import sqlite3
import csv
import os
import shutil
from pathlib import Path

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ Python版本: {pd.__version__}")
print(f"✅ NumPy版本: {np.__version__}")
print(f"✅ CPU核心数: {multiprocessing.cpu_count()}")

# 1. 大数据处理基础概念
print(f"\n📝 1. 大数据处理基础概念:")

# 1.1 创建大数据集
print(f"\n   📊 1.1 创建大数据集:")

def create_big_dataset(n_records: int = 100000, n_features: int = 20):
    """创建用于大数据处理的示例数据集"""
    print(f"   生成大数据集: {n_records:,} 条记录, {n_features} 个特征")
    
    np.random.seed(42)
    
    # 生成基础数据
    data = {
        'id': range(n_records),
        'timestamp': pd.date_range('2020-01-01', periods=n_records, freq='1min'),
        'user_id': np.random.randint(1, 10000, n_records),
        'session_id': np.random.randint(1, 5000, n_records),
        'request_type': np.random.choice(['chat', 'completion', 'embedding', 'classification'], n_records),
        'model_name': np.random.choice(['gpt-3.5-turbo', 'gpt-4', 'claude-3', 'llama-2'], n_records),
        'response_time': np.random.exponential(2.0, n_records) + np.random.normal(0, 0.5, n_records),
        'token_count': np.random.poisson(150, n_records),
        'cost': np.random.uniform(0.001, 0.1, n_records),
        'success_rate': np.random.beta(8, 2, n_records),
        'error_code': np.random.choice([None, 'timeout', 'rate_limit', 'invalid_request'], n_records, p=[0.9, 0.05, 0.03, 0.02]),
        'latency': np.random.gamma(2, 1, n_records),
        'cpu_usage': np.random.beta(2, 5, n_records) * 100,
        'memory_usage': np.random.beta(3, 2, n_records) * 100,
        'network_io': np.random.exponential(10, n_records),
        'disk_io': np.random.exponential(5, n_records),
        'queue_length': np.random.poisson(10, n_records),
        'concurrent_users': np.random.randint(1, 100, n_records),
        'cache_hit_rate': np.random.beta(5, 3, n_records),
        'retry_count': np.random.poisson(0.5, n_records)
    }
    
    # 添加数值特征
    for i in range(n_features - len(data) + 1):
        data[f'feature_{i+1}'] = np.random.normal(0, 1, n_records)
    
    df = pd.DataFrame(data)
    
    # 确保数据合理性
    df['response_time'] = np.maximum(df['response_time'], 0.1)
    df['token_count'] = np.maximum(df['token_count'], 1)
    df['cost'] = np.maximum(df['cost'], 0.001)
    df['success_rate'] = np.clip(df['success_rate'], 0, 1)
    df['latency'] = np.maximum(df['latency'], 0.01)
    df['cpu_usage'] = np.clip(df['cpu_usage'], 0, 100)
    df['memory_usage'] = np.clip(df['memory_usage'], 0, 100)
    df['network_io'] = np.maximum(df['network_io'], 0.1)
    df['disk_io'] = np.maximum(df['disk_io'], 0.1)
    df['queue_length'] = np.maximum(df['queue_length'], 0)
    df['concurrent_users'] = np.maximum(df['concurrent_users'], 1)
    df['cache_hit_rate'] = np.clip(df['cache_hit_rate'], 0, 1)
    df['retry_count'] = np.maximum(df['retry_count'], 0)
    
    # 添加一些相关性
    df['total_cost'] = df['cost'] * df['token_count'] / 100
    df['efficiency'] = df['token_count'] / (df['response_time'] + 0.1)
    df['load_factor'] = (df['cpu_usage'] + df['memory_usage']) / 200
    
    print(f"   数据集创建完成: {df.shape[0]:,} 行 × {df.shape[1]} 列")
    print(f"   内存使用: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return df

# 创建大数据集
big_df = create_big_dataset(n_records=100000, n_features=20)
print(f"   主要指标: API请求、性能指标、系统资源使用等")

# 1.2 大数据处理基础框架
print(f"\n   🔧 1.2 大数据处理基础框架:")

@dataclass
class BigDataProcessor:
    """大数据处理基础框架"""
    
    def __init__(self, df: pd.DataFrame, chunk_size: int = 10000):
        self.df = df.copy()
        self.chunk_size = chunk_size
        self.processing_results = {}
        self.performance_metrics = {}
    
    def basic_info(self):
        """基础信息分析"""
        print(f"   大数据集基本信息:")
        
        info = {
            '总行数': len(self.df),
            '总列数': len(self.df.columns),
            '内存使用(MB)': self.df.memory_usage(deep=True).sum() / 1024**2,
            '数据类型': self.df.dtypes.value_counts().to_dict(),
            '缺失值': self.df.isnull().sum().sum(),
            '重复行': self.df.duplicated().sum()
        }
        
        for key, value in info.items():
            if isinstance(value, dict):
                print(f"      {key}:")
                for k, v in value.items():
                    print(f"        {k}: {v}")
            else:
                print(f"      {key}: {value:,}")
        
        return info
    
    def chunked_processing(self, chunk_func, *args, **kwargs):
        """分块处理大数据"""
        print(f"   分块处理大数据 (块大小: {self.chunk_size:,}):")
        
        start_time = time.time()
        results = []
        
        for i in range(0, len(self.df), self.chunk_size):
            chunk = self.df.iloc[i:i+self.chunk_size]
            chunk_result = chunk_func(chunk, *args, **kwargs)
            results.append(chunk_result)
            
            if (i // self.chunk_size + 1) % 10 == 0:
                print(f"      已处理 {min(i+self.chunk_size, len(self.df)):,} / {len(self.df):,} 行")
        
        processing_time = time.time() - start_time
        print(f"      处理完成，耗时: {processing_time:.2f} 秒")
        
        return results, processing_time
    
    def parallel_processing(self, process_func, n_workers: int = None, 
                            use_processes: bool = True):
        """并行处理大数据"""
        print(f"   并行处理大数据 (工作线程: {n_workers or multiprocessing.cpu_count()}):")
        
        if n_workers is None:
            n_workers = multiprocessing.cpu_count()
        
        # 分割数据
        chunks = [self.df.iloc[i:i+self.chunk_size] 
                 for i in range(0, len(self.df), self.chunk_size)]
        
        start_time = time.time()
        
        if use_processes:
            # 使用进程池
            with ProcessPoolExecutor(max_workers=n_workers) as executor:
                results = list(executor.map(process_func, chunks))
        else:
            # 使用线程池
            with ThreadPoolExecutor(max_workers=n_workers) as executor:
                results = list(executor.map(process_func, chunks))
        
        processing_time = time.time() - start_time
        print(f"      并行处理完成，耗时: {processing_time:.2f} 秒")
        
        return results, processing_time
    
    def memory_optimization(self):
        """内存优化"""
        print(f"   内存优化:")
        
        original_memory = self.df.memory_usage(deep=True).sum() / 1024**2
        print(f"      优化前内存使用: {original_memory:.2f} MB")
        
        # 优化数值类型
        for col in self.df.select_dtypes(include=['int64']).columns:
            col_min = self.df[col].min()
            col_max = self.df[col].max()
            
            if col_min >= 0:
                if col_max < 255:
                    self.df[col] = self.df[col].astype('uint8')
                elif col_max < 65535:
                    self.df[col] = self.df[col].astype('uint16')
                elif col_max < 4294967295:
                    self.df[col] = self.df[col].astype('uint32')
            else:
                if col_min > -128 and col_max < 127:
                    self.df[col] = self.df[col].astype('int8')
                elif col_min > -32768 and col_max < 32767:
                    self.df[col] = self.df[col].astype('int16')
                elif col_min > -2147483648 and col_max < 2147483647:
                    self.df[col] = self.df[col].astype('int32')
        
        # 优化浮点类型
        for col in self.df.select_dtypes(include=['float64']).columns:
            self.df[col] = pd.to_numeric(self.df[col], downcast='float')
        
        # 优化字符串类型
        for col in self.df.select_dtypes(include=['object']).columns:
            if self.df[col].nunique() / len(self.df) < 0.5:  # 如果重复率较高
                self.df[col] = self.df[col].astype('category')
        
        optimized_memory = self.df.memory_usage(deep=True).sum() / 1024**2
        memory_reduction = (original_memory - optimized_memory) / original_memory * 100
        
        print(f"      优化后内存使用: {optimized_memory:.2f} MB")
        print(f"      内存减少: {memory_reduction:.1f}%")
        
        return {
            'original_memory': original_memory,
            'optimized_memory': optimized_memory,
            'memory_reduction': memory_reduction
        }
    
    def data_sampling(self, method: str = 'random', sample_size: float = 0.1):
        """数据采样"""
        print(f"   数据采样 ({method}, 比例: {sample_size}):")
        
        n_samples = int(len(self.df) * sample_size)
        
        if method == 'random':
            sample = self.df.sample(n=n_samples, random_state=42)
        elif method == 'systematic':
            step = len(self.df) // n_samples
            sample = self.df.iloc[::step][:n_samples]
        elif method == 'stratified':
            # 按request_type分层采样
            sample = self.df.groupby('request_type').apply(
                lambda x: x.sample(n=min(len(x), n_samples // len(self.df['request_type'].unique())), random_state=42)
            ).reset_index(drop=True)
        else:
            raise ValueError("不支持的采样方法")
        
        print(f"      采样完成: {len(sample):,} 条记录")
        print(f"      采样比例: {len(sample)/len(self.df):.3f}")
        
        return sample

# 初始化大数据处理框架
big_processor = BigDataProcessor(big_df, chunk_size=10000)
print(f"   ✅ 大数据处理框架初始化完成")

# 基础信息分析
basic_info = big_processor.basic_info()

print(f"\n✅ 大数据处理基础概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握大数据处理的基本概念")
print(f"   ✓ 理解大数据处理的挑战")
print(f"   ✓ 熟悉大数据处理框架")
print(f"   ✓ 能够创建和处理大数据集")

### 分布式计算技术 [⭐⭐进阶]
**知识点说明**：分布式计算是大数据处理的核心技术，包括MapReduce、并行处理、任务调度等。掌握这些算法对于构建高性能LangChain应用非常重要。

**学习要求**：
- 掌握MapReduce的基本原理
- 理解并行计算模式
- 熟练使用分布式计算工具
- 能够优化分布式计算性能

**案例要求**：
- 实现完整的分布式计算流程
- 进行并行计算的性能优化
- 应用分布式计算解决实际问题
- 验证点：能独立构建有效的分布式计算系统

In [ ]:
print("\n🚀 分布式计算技术:")
print("=" * 50)

# 1. MapReduce实现
print(f"📝 1. MapReduce实现:")

@dataclass
class MapReduceFramework:
    """MapReduce计算框架实现"""
    
    def __init__(self, data: List[Any]):
        self.data = data
        self.map_results = []
        self.reduce_results = {}
    
    def map_function(self, mapper_func):
        """Map阶段"""
        print(f"   Map阶段处理 {len(self.data):,} 条数据...")
        
        start_time = time.time()
        self.map_results = []
        
        for item in self.data:
            mapped_item = mapper_func(item)
            self.map_results.extend(mapped_item)
        
        map_time = time.time() - start_time
        print(f"      Map完成，生成 {len(self.map_results):,} 个键值对，耗时: {map_time:.3f} 秒")
        
        return self.map_results, map_time
    
    def shuffle_sort(self):
        """Shuffle和Sort阶段"""
        print(f"   Shuffle和Sort阶段...")
        
        start_time = time.time()
        
        # 按键分组
        grouped_data = {}
        for key, value in self.map_results:
            if key not in grouped_data:
                grouped_data[key] = []
            grouped_data[key].append(value)
        
        shuffle_time = time.time() - start_time
        print(f"      Shuffle完成，分组为 {len(grouped_data):,} 个键，耗时: {shuffle_time:.3f} 秒")
        
        return grouped_data, shuffle_time
    
    def reduce_function(self, reducer_func, grouped_data):
        """Reduce阶段"""
        print(f"   Reduce阶段处理 {len(grouped_data):,} 个分组...")
        
        start_time = time.time()
        self.reduce_results = {}
        
        for key, values in grouped_data.items():
            reduced_value = reducer_func(key, values)
            self.reduce_results[key] = reduced_value
        
        reduce_time = time.time() - start_time
        print(f"      Reduce完成，生成 {len(self.reduce_results):,} 个结果，耗时: {reduce_time:.3f} 秒")
        
        return self.reduce_results, reduce_time
    
    def execute(self, mapper_func, reducer_func):
        """执行完整的MapReduce流程"""
        total_start = time.time()
        
        # Map阶段
        map_results, map_time = self.map_function(mapper_func)
        
        # Shuffle阶段
        grouped_data, shuffle_time = self.shuffle_sort()
        
        # Reduce阶段
        reduce_results, reduce_time = self.reduce_function(reducer_func, grouped_data)
        
        total_time = time.time() - total_start
        
        print(f"\n   📊 MapReduce执行统计:")
        print(f"      总耗时: {total_time:.3f} 秒")
        print(f"      Map阶段: {map_time:.3f} 秒 ({map_time/total_time*100:.1f}%)")
        print(f"      Shuffle阶段: {shuffle_time:.3f} 秒 ({shuffle_time/total_time*100:.1f}%)")
        print(f"      Reduce阶段: {reduce_time:.3f} 秒 ({reduce_time/total_time*100:.1f}%)")
        
        return {
            'results': reduce_results,
            'timing': {
                'total': total_time,
                'map': map_time,
                'shuffle': shuffle_time,
                'reduce': reduce_time
            }
        }

# 2. 并行计算框架
print(f"\n   🔧 2. 并行计算框架:")

@dataclass
class ParallelComputing:
    """并行计算框架实现"""
    
    def __init__(self, data: List[Any]):
        self.data = data
        self.n_workers = multiprocessing.cpu_count()
    
    def parallel_map(self, func, chunk_size: int = None):
        """并行Map操作"""
        print(f"   并行Map操作 (工作进程: {self.n_workers}):")
        
        if chunk_size is None:
            chunk_size = max(1, len(self.data) // self.n_workers)
        
        # 分割数据
        chunks = [self.data[i:i+chunk_size] 
                 for i in range(0, len(self.data), chunk_size)]
        
        print(f"      数据分割为 {len(chunks)} 个块，每块约 {chunk_size:,} 条数据")
        
        start_time = time.time()
        
        with ProcessPoolExecutor(max_workers=self.n_workers) as executor:
            chunk_results = list(executor.map(func, chunks))
        
        # 合并结果
        results = []
        for chunk_result in chunk_results:
            results.extend(chunk_result)
        
        parallel_time = time.time() - start_time
        print(f"      并行处理完成，耗时: {parallel_time:.3f} 秒")
        
        return results, parallel_time
    
    def sequential_map(self, func):
        """串行Map操作（用于性能对比）"""
        print(f"   串行Map操作:")
        
        start_time = time.time()
        results = func(self.data)
        sequential_time = time.time() - start_time
        
        print(f"      串行处理完成，耗时: {sequential_time:.3f} 秒")
        
        return results, sequential_time
    
    def performance_comparison(self, func, chunk_size: int = None):
        """性能对比"""
        print(f"\n   📊 并行vs串行性能对比:")
        
        # 串行处理
        sequential_results, sequential_time = self.sequential_map(func)
        
        # 并行处理
        parallel_results, parallel_time = self.parallel_map(func, chunk_size)
        
        # 验证结果一致性
        results_match = len(sequential_results) == len(parallel_results)
        
        # 计算加速比
        speedup = sequential_time / parallel_time if parallel_time > 0 else float('inf')
        efficiency = speedup / self.n_workers * 100
        
        print(f"      数据量: {len(self.data):,}")
        print(f"      工作进程: {self.n_workers}")
        print(f"      串行时间: {sequential_time:.3f} 秒")
        print(f"      并行时间: {parallel_time:.3f} 秒")
        print(f"      加速比: {speedup:.2f}x")
        print(f"      并行效率: {efficiency:.1f}%")
        print(f"      结果一致性: {'✓' if results_match else '✗'}")
        
        return {
            'sequential_time': sequential_time,
            'parallel_time': parallel_time,
            'speedup': speedup,
            'efficiency': efficiency,
            'results_match': results_match
        }

# 3. 流式处理框架
print(f"\n   🌊 3. 流式处理框架:")

@dataclass
class StreamProcessor:
    """流式处理框架实现"""
    
    def __init__(self, buffer_size: int = 1000):
        self.buffer_size = buffer_size
        self.buffer = queue.Queue(maxsize=buffer_size)
        self.processed_count = 0
        self.error_count = 0
    
    def data_generator(self, data_source, delay: float = 0.001):
        """数据生成器"""
        for item in data_source:
            time.sleep(delay)  # 模拟数据到达延迟
            yield item
    
    def stream_filter(self, predicate):
        """流式过滤"""
        def filter_generator(data_gen):
            for item in data_gen:
                if predicate(item):
                    yield item
        return filter_generator
    
    def stream_map(self, transform_func):
        """流式映射"""
        def map_generator(data_gen):
            for item in data_gen:
                try:
                    transformed = transform_func(item)
                    yield transformed
                    self.processed_count += 1
                except Exception as e:
                    self.error_count += 1
                    print(f"      处理错误: {e}")
        return map_generator
    
    def stream_reduce(self, reduce_func, window_size: int = 100):
        """流式聚合（滑动窗口）"""
        def reduce_generator(data_gen):
            window = []
            for item in data_gen:
                window.append(item)
                if len(window) >= window_size:
                    result = reduce_func(window)
                    yield result
                    window = window[1:]  # 滑动窗口
            
            # 处理剩余数据
            if window:
                yield reduce_func(window)
        
        return reduce_generator
    
    def process_stream(self, data_source, processors):
        """处理数据流"""
        print(f"   开始流式处理 (缓冲区: {self.buffer_size}):")
        
        start_time = time.time()
        
        # 创建数据生成器
        data_gen = self.data_generator(data_source)
        
        # 应用处理器链
        for processor in processors:
            data_gen = processor(data_gen)
        
        # 收集结果
        results = []
        for result in data_gen:
            results.append(result)
            
            if len(results) % 100 == 0:
                print(f"      已处理 {len(results):,} 个结果")
        
        processing_time = time.time() - start_time
        
        print(f"      流式处理完成:")
        print(f"        处理时间: {processing_time:.3f} 秒")
        print(f"        成功处理: {self.processed_count:,}")
        print(f"        错误数量: {self.error_count:,}")
        print(f"        输出结果: {len(results):,}")
        
        return {
            'results': results,
            'processing_time': processing_time,
            'processed_count': self.processed_count,
            'error_count': self.error_count
        }

# 演示分布式计算技术
print(f"\n   🔧 分布式计算技术演示:")

# 1. 准备测试数据
print(f"\n   📊 准备测试数据:")
test_data = [
    {'text': f'langchain is great for ai development {i}', 
     'category': np.random.choice(['tech', 'business', 'science']), 
     'score': np.random.uniform(0, 1)}
    for i in range(10000)
]
print(f"   创建了 {len(test_data):,} 条测试数据")

# 2. MapReduce演示
print(f"\n   🎯 MapReduce演示:")

# 定义mapper函数：词频统计
def word_count_mapper(document):
    words = document['text'].lower().split()
    return [(word, 1) for word in words]

# 定义reducer函数：词频汇总
def word_count_reducer(word, counts):
    return (word, sum(counts))

# 执行MapReduce
mr_framework = MapReduceFramework(test_data)
mr_results = mr_framework.execute(word_count_mapper, word_count_reducer)

# 显示前10个最常见词
top_words = sorted(mr_results['results'].items(), key=lambda x: x[1], reverse=True)[:10]
print(f"\n   📈 词频统计Top10:")
for word, count in top_words:
    print(f"      {word}: {count}")

# 3. 并行计算演示
print(f"\n   🚀 并行计算演示:")

# 定义处理函数
def process_documents_chunk(documents):
    results = []
    for doc in documents:
        # 模拟复杂处理
        processed = {
            'id': hash(doc['text']) % 1000000,
            'length': len(doc['text']),
            'category': doc['category'],
            'score_normalized': doc['score'] / 1.0,
            'word_count': len(doc['text'].split())
        }
        results.append(processed)
    return results

# 性能对比
parallel_comp = ParallelComputing(test_data)
performance_results = parallel_comp.performance_comparison(process_documents_chunk)

# 4. 流式处理演示
print(f"\n   🌊 流式处理演示:")

# 定义处理函数
def extract_features(item):
    return {
        'category': item['category'],
        'score_bucket': int(item['score'] * 5),
        'text_length': len(item['text'])
    }

def aggregate_features(items):
    categories = [item['category'] for item in items]
    scores = [item['score_bucket'] for item in items]
    lengths = [item['text_length'] for item in items]
    
    return {
        'count': len(items),
        'avg_score': np.mean(scores),
        'avg_length': np.mean(lengths),
        'category_distribution': pd.Series(categories).value_counts().to_dict()
    }

# 流式处理
stream_processor = StreamProcessor(buffer_size=1000)
processors = [
    stream_processor.stream_filter(lambda x: x['score'] > 0.5),  # 过滤高分数据
    stream_processor.stream_map(extract_features),  # 特征提取
    stream_processor.stream_reduce(aggregate_features, window_size=50)  # 聚合
]

stream_results = stream_processor.process_stream(test_data[:1000], processors)

print(f"\n✅ 分布式计算技术完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握MapReduce的基本原理")
print(f"   ✓ 理解并行计算模式")
print(f"   ✓ 熟练使用分布式计算工具")
print(f"   ✓ 能够优化分布式计算性能")

### 大数据存储与检索 [⭐⭐进阶]
**知识点说明**：大数据存储与检索是大数据处理的基础，包括分布式存储、索引技术、缓存策略等。掌握这些技术对于构建高性能LangChain应用非常重要。

**学习要求**：
- 掌握分布式存储的基本原理
- 理解数据索引和检索技术
- 熟练使用缓存和优化策略
- 能够设计高效的数据存储方案

**案例要求**：
- 实现完整的数据存储和检索流程
- 进行数据访问的性能优化
- 应用存储技术解决实际问题
- 验证点：能独立构建高效的数据存储系统

In [ ]:
print("\n💾 大数据存储与检索:")
print("=" * 50)

# 1. 分布式存储模拟
print(f"📝 1. 分布式存储模拟:")

@dataclass
class DistributedStorage:
    """分布式存储系统模拟"""
    
    def __init__(self, n_nodes: int = 3, replication_factor: int = 2):
        self.n_nodes = n_nodes
        self.replication_factor = replication_factor
        self.nodes = [{} for _ in range(n_nodes)]
        self.key_locations = {}
        self.access_stats = {i: {'reads': 0, 'writes': 0} for i in range(n_nodes)}
    
    def hash_key(self, key: str) -> int:
        """一致性哈希"""
        return int(hashlib.md5(key.encode()).hexdigest(), 16) % self.n_nodes
    
    def get_replica_nodes(self, primary_node: int) -> List[int]:
        """获取副本节点"""
        replicas = [primary_node]
        for i in range(1, self.replication_factor):
            replica_node = (primary_node + i) % self.n_nodes
            replicas.append(replica_node)
        return replicas
    
    def put(self, key: str, value: Any):
        """存储数据"""
        primary_node = self.hash_key(key)
        replica_nodes = self.get_replica_nodes(primary_node)
        
        # 存储到所有副本节点
        for node_id in replica_nodes:
            self.nodes[node_id][key] = value
            self.access_stats[node_id]['writes'] += 1
        
        self.key_locations[key] = replica_nodes
        
        return replica_nodes
    
    def get(self, key: str) -> Any:
        """获取数据"""
        if key not in self.key_locations:
            raise KeyError(f"Key {key} not found")
        
        # 从主节点读取
        primary_node = self.key_locations[key][0]
        self.access_stats[primary_node]['reads'] += 1
        
        return self.nodes[primary_node].get(key)
    
    def delete(self, key: str):
        """删除数据"""
        if key not in self.key_locations:
            return False
        
        # 从所有副本节点删除
        for node_id in self.key_locations[key]:
            if key in self.nodes[node_id]:
                del self.nodes[node_id][key]
        
        del self.key_locations[key]
        return True
    
    def get_storage_stats(self):
        """获取存储统计"""
        stats = {
            'total_keys': len(self.key_locations),
            'node_stats': []
        }
        
        for i in range(self.n_nodes):
            node_stats = {
                'node_id': i,
                'key_count': len(self.nodes[i]),
                'reads': self.access_stats[i]['reads'],
                'writes': self.access_stats[i]['writes'],
                'memory_usage': len(str(self.nodes[i]))
            }
            stats['node_stats'].append(node_stats)
        
        return stats
    
    def rebalance(self):
        """数据重平衡"""
        print(f"   执行数据重平衡...")
        
        # 收集所有数据
        all_data = {}
        for node in self.nodes:
            all_data.update(node)
        
        # 清空所有节点
        self.nodes = [{} for _ in range(self.n_nodes)]
        self.key_locations = {}
        
        # 重新分布数据
        for key, value in all_data.items():
            self.put(key, value)
        
        print(f"      重平衡完成，重新分布 {len(all_data)} 个键")

# 2. 索引和检索系统
print(f"\n   🔍 2. 索引和检索系统:")

@dataclass
class IndexSystem:
    """索引和检索系统"""
    
    def __init__(self):
        self.documents = {}
        self.inverted_index = {}  # 倒排索引
        self.field_index = {}     # 字段索引
        self.search_stats = {'searches': 0, 'results': 0}
    
    def add_document(self, doc_id: str, document: Dict[str, Any]):
        """添加文档"""
        self.documents[doc_id] = document
        
        # 构建文本倒排索引
        if 'text' in document:
            words = document['text'].lower().split()
            for word in set(words):  # 去重
                if word not in self.inverted_index:
                    self.inverted_index[word] = []
                self.inverted_index[word].append(doc_id)
        
        # 构建字段索引
        for field, value in document.items():
            if field != 'text':  # 文本字段已单独处理
                if field not in self.field_index:
                    self.field_index[field] = {}
                
                field_value = str(value).lower()
                if field_value not in self.field_index[field]:
                    self.field_index[field][field_value] = []
                self.field_index[field][field_value].append(doc_id)
    
    def search_text(self, query: str, operator: str = 'OR') -> List[str]:
        """文本搜索"""
        self.search_stats['searches'] += 1
        
        query_words = query.lower().split()
        result_sets = []
        
        for word in query_words:
            if word in self.inverted_index:
                result_sets.append(set(self.inverted_index[word]))
        
        if not result_sets:
            return []
        
        # 合并结果
        if operator == 'OR':
            results = set.union(*result_sets)
        elif operator == 'AND':
            results = set.intersection(*result_sets)
        else:
            raise ValueError("不支持的操作符")
        
        self.search_stats['results'] += len(results)
        return list(results)
    
    def search_field(self, field: str, value: str) -> List[str]:
        """字段搜索"""
        self.search_stats['searches'] += 1
        
        if field not in self.field_index:
            return []
        
        search_value = str(value).lower()
        results = self.field_index[field].get(search_value, [])
        
        self.search_stats['results'] += len(results)
        return results
    
    def boolean_search(self, query: str) -> List[str]:
        """布尔搜索"""
        # 简化的布尔搜索实现
        # 支持格式: "word1 AND word2" 或 "word1 OR word2"
        
        if ' AND ' in query:
            parts = query.split(' AND ')
            return self.search_text(' '.join(parts), operator='AND')
        elif ' OR ' in query:
            parts = query.split(' OR ')
            return self.search_text(' '.join(parts), operator='OR')
        else:
            return self.search_text(query)
    
    def get_index_stats(self):
        """获取索引统计"""
        stats = {
            'total_documents': len(self.documents),
            'inverted_index_size': len(self.inverted_index),
            'field_index_size': len(self.field_index),
            'search_stats': self.search_stats.copy(),
            'avg_results_per_search': (
                self.search_stats['results'] / max(1, self.search_stats['searches'])
            )
        }
        return stats

# 3. 缓存系统
print(f"\n   ⚡ 3. 缓存系统:")

@dataclass
class CacheSystem:
    """缓存系统实现"""
    
    def __init__(self, max_size: int = 1000, policy: str = 'LRU'):
        self.max_size = max_size
        self.policy = policy
        self.cache = {}
        self.access_order = []  # 用于LRU
        self.access_count = {}  # 用于LFU
        self.stats = {
            'hits': 0,
            'misses': 0,
            'evictions': 0
        }
    
    def get(self, key: str) -> Any:
        """获取缓存值"""
        if key in self.cache:
            self.stats['hits'] += 1
            
            # 更新访问信息
            if self.policy == 'LRU':
                self.access_order.remove(key)
                self.access_order.append(key)
            elif self.policy == 'LFU':
                self.access_count[key] += 1
            
            return self.cache[key]
        else:
            self.stats['misses'] += 1
            return None
    
    def put(self, key: str, value: Any):
        """存储缓存值"""
        if key in self.cache:
            # 更新现有值
            self.cache[key] = value
            
            if self.policy == 'LRU':
                self.access_order.remove(key)
                self.access_order.append(key)
            elif self.policy == 'LFU':
                self.access_count[key] += 1
        else:
            # 添加新值
            if len(self.cache) >= self.max_size:
                self._evict()
            
            self.cache[key] = value
            
            if self.policy == 'LRU':
                self.access_order.append(key)
            elif self.policy == 'LFU':
                self.access_count[key] = 1
    
    def _evict(self):
        """驱逐缓存项"""
        if not self.cache:
            return
        
        if self.policy == 'LRU':
            # 驱逐最近最少使用的项
            lru_key = self.access_order[0]
            del self.cache[lru_key]
            self.access_order.remove(lru_key)
        elif self.policy == 'LFU':
            # 驱逐使用频率最低的项
            lfu_key = min(self.access_count.keys(), key=lambda k: self.access_count[k])
            del self.cache[lfu_key]
            del self.access_count[lfu_key]
        
        self.stats['evictions'] += 1
    
    def clear(self):
        """清空缓存"""
        self.cache.clear()
        self.access_order.clear()
        self.access_count.clear()
    
    def get_stats(self):
        """获取缓存统计"""
        total_access = self.stats['hits'] + self.stats['misses']
        hit_rate = self.stats['hits'] / max(1, total_access)
        
        return {
            'size': len(self.cache),
            'max_size': self.max_size,
            'hit_rate': hit_rate,
            'hits': self.stats['hits'],
            'misses': self.stats['misses'],
            'evictions': self.stats['evictions'],
            'policy': self.policy
        }

# 演示大数据存储与检索
print(f"\n   🔧 大数据存储与检索演示:")

# 1. 分布式存储演示
print(f"\n   💾 分布式存储演示:")

# 创建分布式存储系统
dist_storage = DistributedStorage(n_nodes=3, replication_factor=2)

# 存储测试数据
test_keys = [f'doc_{i}' for i in range(100)]
for i, key in enumerate(test_keys):
    value = {
        'content': f'This is document {i} content',
        'timestamp': time.time(),
        'size': np.random.randint(100, 1000)
    }
    dist_storage.put(key, value)

# 测试读写
print(f"   测试数据读写...")
for key in test_keys[:5]:
    data = dist_storage.get(key)
    print(f"      {key}: {len(data['content'])} 字符")

# 显示存储统计
storage_stats = dist_storage.get_storage_stats()
print(f"\n   📊 分布式存储统计:")
print(f"      总键数: {storage_stats['total_keys']}")
for node_stat in storage_stats['node_stats']:
    print(f"      节点{node_stat['node_id']}: {node_stat['key_count']} 键, "
          f"读{node_stat['reads']} 写{node_stat['writes']}")

# 2. 索引系统演示
print(f"\n   🔍 索引系统演示:")

# 创建索引系统
index_system = IndexSystem()

# 添加文档
documents = [
    {'doc_id': f'doc_{i}', 
     'text': f'LangChain is a framework for building applications powered by language models document {i}',
     'category': np.random.choice(['AI', 'ML', 'NLP']),
     'author': f'author_{i % 5}'}
    for i in range(50)
]

for doc in documents:
    index_system.add_document(doc['doc_id'], doc)

# 测试搜索
print(f"   测试文本搜索...")
search_queries = [
    'LangChain',
    'language models',
    'LangChain AND framework',
    'AI OR ML'
]

for query in search_queries:
    results = index_system.boolean_search(query)
    print(f"      '{query}': {len(results)} 个结果")

# 测试字段搜索
print(f"\n   测试字段搜索...")
field_results = index_system.search_field('category', 'AI')
print(f"      类别'AI': {len(field_results)} 个结果")

# 显示索引统计
index_stats = index_system.get_index_stats()
print(f"\n   📊 索引系统统计:")
print(f"      总文档数: {index_stats['total_documents']}")
print(f"      倒排索引大小: {index_stats['inverted_index_size']}")
print(f"      字段索引大小: {index_stats['field_index_size']}")
print(f"      平均搜索结果数: {index_stats['avg_results_per_search']:.1f}")

# 3. 缓存系统演示
print(f"\n   ⚡ 缓存系统演示:")

# 创建缓存系统
cache_lru = CacheSystem(max_size=50, policy='LRU')
cache_lfu = CacheSystem(max_size=50, policy='LFU')

# 模拟缓存访问模式
def simulate_cache_access(cache, access_pattern):
    """模拟缓存访问"""
    for key in access_pattern:
        # 尝试获取
        value = cache.get(key)
        
        # 如果未命中，则加载并缓存
        if value is None:
            value = f'data_for_{key}'
            cache.put(key, value)

# 生成访问模式（模拟真实场景）
np.random.seed(42)
access_pattern = []
# 80%的访问集中在20%的热点数据
hot_keys = [f'key_{i}' for i in range(20)]
cold_keys = [f'key_{i}' for i in range(20, 100)]

for _ in range(200):
    if np.random.random() < 0.8:
        access_pattern.append(np.random.choice(hot_keys))
    else:
        access_pattern.append(np.random.choice(cold_keys))

# 测试LRU缓存
print(f"   测试LRU缓存策略...")
simulate_cache_access(cache_lru, access_pattern)
lru_stats = cache_lru.get_stats()

# 测试LFU缓存
print(f"   测试LFU缓存策略...")
simulate_cache_access(cache_lfu, access_pattern)
lfu_stats = cache_lfu.get_stats()

# 缓存性能对比
print(f"\n   📊 缓存性能对比:")
print(f"      策略     命中率   命中数   未命中数   驱逐数")
print(f"      LRU      {lru_stats['hit_rate']:.3f}   {lru_stats['hits']:4d}    {lru_stats['misses']:4d}      {lru_stats['evictions']:3d}")
print(f"      LFU      {lfu_stats['hit_rate']:.3f}   {lfu_stats['hits']:4d}    {lfu_stats['misses']:4d}      {lfu_stats['evictions']:3d}")

print(f"\n✅ 大数据存储与检索完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握分布式存储的基本原理")
print(f"   ✓ 理解数据索引和检索技术")
print(f"   ✓ 熟练使用缓存和优化策略")
print(f"   ✓ 能够设计高效的数据存储方案")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**5.8 大数据处理技术 [⭐⭐进阶]**
- ✅ 掌握大数据处理的基本概念
- ✅ 理解分布式计算原理
- ✅ 熟练使用大数据处理工具
- ✅ 能够设计可扩展的数据处理架构
- ✅ 掌握MapReduce的基本原理
- ✅ 理解并行计算模式
- ✅ 熟练使用分布式计算工具
- ✅ 能够优化分布式计算性能
- ✅ 掌握分布式存储的基本原理
- ✅ 理解数据索引和检索技术
- ✅ 熟练使用缓存和优化策略
- ✅ 能够设计高效的数据存储方案
- ✅ 能独立构建有效的大数据处理系统

### 🎯 与LangChain学习的关联

**大数据处理重要性**：
- 大数据处理支持LangChain的海量文本数据处理
- 为LangChain的分布式推理提供技术基础
- 支持LangChain的大规模知识库管理
- 确保LangChain应用的高性能和可扩展性
- 大数据处理支持LangChain的实时分析和决策

**实际应用场景**：
- LangChain的大规模文档处理和向量化
- LangChain的分布式模型推理和计算
- LangChain的实时对话流处理和分析
- LangChain的知识图谱构建和查询优化
- LangChain的用户行为分析和个性化推荐

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习Apache Spark和Dask等分布式框架
   - 掌握流式处理框架如Apache Kafka和Flink
   - 学习云原生大数据平台如AWS EMR、Google Dataflow

2. **扩展学习**：
   - 学习分布式数据库和NoSQL技术
   - 了解数据湖和数据仓库架构
   - 探索实时计算和复杂事件处理

3. **实际应用**：
   - 构建企业级大数据处理平台
   - 开发实时数据流处理系统
   - 实现智能数据管道和ETL流程

### 🔧 常见错误与注意事项

1. **内存管理错误**：
   ```python
   # 错误：一次性加载全部大数据
   big_data = pd.read_csv('huge_file.csv')  # 内存溢出
   
   # 正确：分块加载和处理
   chunk_size = 10000
   for chunk in pd.read_csv('huge_file.csv', chunksize=chunk_size):
       process_chunk(chunk)
   ```

2. **并行计算误区**：
   ```python
   # 错误：过度并行化导致开销过大
   with ProcessPoolExecutor(max_workers=100) as executor:  # 超过CPU核心数
       results = list(executor.map(func, data))
   
   # 正确：根据硬件资源合理设置并行度
   n_workers = min(multiprocessing.cpu_count(), len(data) // chunk_size)
   with ProcessPoolExecutor(max_workers=n_workers) as executor:
       results = list(executor.map(func, data))
   ```

3. **缓存策略误用**：
   ```python
   # 错误：缓存所有数据导致内存不足
   cache.put('huge_data', massive_dataset)  # 缓存过大数据
   
   # 正确：合理设置缓存大小和驱逐策略
   cache = CacheSystem(max_size=1000, policy='LRU')
   # 只缓存热点数据
   if is_hot_data(key):
       cache.put(key, value)
   ```

4. **数据一致性忽略**：
   ```python
   # 错误：分布式环境下忽略数据一致性
   def update_data(key, value):
       node = get_random_node()  # 随机选择节点
       node[key] = value  # 可能导致数据不一致
   
   # 正确：确保数据一致性和副本同步
   def update_data(key, value):
       primary_node = hash_key(key)
       replica_nodes = get_replica_nodes(primary_node)
       for node in replica_nodes:
           node[key] = value  # 同步更新所有副本
   ```

5. **索引设计不当**：
   ```python
   # 错误：为所有字段创建索引导致性能下降
   for field in all_fields:
       create_index(field)  # 过度索引
   
   # 正确：只为常用查询字段创建索引
   frequent_query_fields = ['title', 'category', 'timestamp']
   for field in frequent_query_fields:
       if is_queried_frequently(field):
           create_index(field)
   ```

6. **流式处理阻塞**：
   ```python
   # 错误：流式处理中使用阻塞操作
   def process_stream(data_stream):
       for item in data_stream:
           result = slow_blocking_operation(item)  # 阻塞整个流
           yield result
   
   # 正确：使用异步或并行处理避免阻塞
   async def process_stream(data_stream):
       async for item in data_stream:
           result = await async_operation(item)
           yield result
   ```

### 🌐 性能优化建议

**大数据处理优化**：
- 使用数据分片和分区提高并行度
- 采用内存计算和列式存储优化性能
- 实现智能缓存和预加载策略
- 使用压缩和编码减少存储和传输开销

**分布式计算优化**：
- 优化数据局部性减少网络传输
- 实现动态负载均衡和任务调度
- 使用容错和故障恢复机制
- 实现监控和性能调优系统

**存储检索优化**：
- 设计合理的索引策略和查询优化
- 使用分层存储和冷热数据分离
- 实现数据压缩和去重技术
- 建立完整的备份和恢复机制

---

**🎉 恭喜完成大数据处理技术学习！**

你已经掌握了大数据处理的核心技能，能够系统性地进行分布式计算、存储优化和流式处理，为LangChain智能应用提供了强大的大数据处理能力。

## 🚀 下一步学习预告

**继续第五节：数据处理**
- 5.9 数据工程实践

**后续章节预告**：
- 异步编程技术
- Web开发技术
- 项目工程实践

继续加油，大数据处理技能正在快速提升！